In [24]:
import os
import datetime
import pandas as pd
import numpy as np
from pathlib import Path

import statsmodels.api as sm

import plotly
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
%matplotlib inline
init_notebook_mode(connected=True)

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score
from sklearn.cross_validation import KFold
from sklearn.pipeline import make_pipeline

/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.



### Data Import & Preparation

In [4]:
full_data = pd.read_csv('data/ml_class_data_ver1.csv')
full_data_with_trends = pd.read_csv('data/ml_class_data_with_trends_ver1.csv')

In [5]:
full_data.set_index(full_data.time, drop=True, inplace=True)
full_data.drop(columns=['time', 'Unnamed: 0'], inplace=True)
full_data_with_trends.set_index(full_data_with_trends.time, drop=True, inplace=True)
full_data_with_trends.drop(columns=['time', 'Unnamed: 0'], inplace=True)

In [6]:
# full_data.head()

In [7]:
target_full = full_data['sign_change']
features_full = full_data.drop(columns = ['sign_change'])

target_with_trends_full = full_data_with_trends['sign_change']
features_with_trends_full = full_data_with_trends.drop(columns = ['sign_change'])

In [45]:
# features_with_trends.head()

In [46]:
test_cut = int(len(target_full) * .1)

target_test = target_full[-test_cut:]
features_test = features_full[-test_cut:]

target_with_trends_test = target_full[-test_cut:]
features_with_trends_test = features_full[-test_cut:]

In [47]:
# features_test

In [48]:
end_cut = len(target_full)-len(target_test)
start_cut = end_cut-test_cut

target_val = target_full[start_cut:end_cut]
features_val = features_full[start_cut:end_cut]

target_with_trends_val = target_with_trends_full[start_cut:end_cut]
features_with_trends_val = features_with_trends_full[start_cut:end_cut]

In [49]:
cut_off = start_cut

target_train = target_full[:cut_off]
features_train = features_full[:cut_off]

target_with_trends_train = target_with_trends_full[:cut_off]
features_with_trends_train = features_with_trends_full[:cut_off]

print(len(target_full))
print(len(target_train)+len(target_test)+len(target_val))

13249
13249


In [50]:
# testing = np.linspace(0, 100, 100)
# df_test = pd.DataFrame(testing)
# test = df_test[-10:]
# train = df_test[:90]

In [51]:
# features.head()

In [52]:
training_sets = {'train': [target_train, features_train, target_val, features_val], 
                'train_with_trend' : [target_with_trends_train, features_with_trends_train, 
                                      target_with_trends_val, features_with_trends_val]}

# training_sets = {'train': ['target_train', 'features_train', 'target_val', 'features_val'], 
#                 'train_with_trend' : ['target_with_trends_train', 'features_with_trends_train', 
#                                       'target_with_trends_val', 'features_with_trends_val']}

In [56]:
for k, v in training_sets.items():
    print(len(v[3]))

1324
1324


### Functions

In [53]:
def test_classifiers(data, grid):
    results_dict = {}
    
    for data_set, splits in data.items():
        grid.fit(splits[1], splits[0])
        results_dict[data_set + ' results'] = [grid.best_score_, grid.best_params_, grid.score(splits[1], splits[0]), grid.score(splits[3], splits[2])]
    
    return results_dict

## Machine Learning

#### Naive Bayes Models - Not implemented, copied from another project. ToDo

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

BernoulliNB

In [ ]:
bernoulli_clf = BernoulliNB()
bernoulli_clf.fit(nb_X_train_sex, nb_y_train_sex)

bernoulli_train_preds = bernoulli_clf.predict(nb_X_train_sex)
bernoulli_test_preds = bernoulli_clf.predict(nb_X_test_sex)

bernoulli_train_accuracy = accuracy_score(nb_y_train_sex, bernoulli_train_preds)
bernoulli_test_accuracy = accuracy_score(nb_y_test_sex, bernoulli_test_preds)

print("Accuracy on Training Set from BernoulliNB Classifier: {:.4}%".format(bernoulli_train_accuracy * 100))
print("Accuracy on Testing Set from BernoulliNB Classifier: {:.4}%".format(bernoulli_test_accuracy * 100))

GaussianNB

In [ ]:
gaussian_clf = GaussianNB()
gaussian_clf.fit(nb_X_train_sex.toarray(), nb_y_train_sex)

gaussian_train_preds = gaussian_clf.predict(nb_X_train_sex.toarray())
gaussian_test_preds = gaussian_clf.predict(nb_X_test_sex.toarray())

gaussian_train_accuracy = accuracy_score(nb_y_train_sex, gaussian_train_preds)
gaussian_test_accuracy = accuracy_score(nb_y_test_sex, gaussian_test_preds)

print("Accuracy on Training Set from GaussianNB Classifier: {:.4}%".format(gaussian_train_accuracy * 100))
print("Accuracy on Testing Set from GaussianNB Classifier::  {:.4}%".format(gaussian_test_accuracy * 100))

MultinomialNB

In [ ]:
mnb_clf = MultinomialNB()
mnb_clf.fit(nb_X_train_sex, nb_y_train_sex)

mnb_train_preds = mnb_clf.predict(nb_X_train_sex)
mnb_test_preds = mnb_clf.predict(nb_X_test_sex)

mnb_train_accuracy = accuracy_score(nb_y_train_sex, mnb_train_preds)
mnb_test_accuracy = accuracy_score(nb_y_test_sex, mnb_test_preds)

print("Accuracy on Training Set from MultinomialNB Classifier: {:.4}%".format(mnb_train_accuracy * 100))
print("Accuracy on Testing Set from MultinomialNB Classifier:: {:.4}%".format(mnb_test_accuracy * 100))

#### Logistic Regression 

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
lgr_param_grid = {'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
                 'logisticregression__penalty' : ['l1', 'l2']}

lgr_pipe = make_pipeline(LogisticRegression())

lgr_grid = GridSearchCV(lgr_pipe, lgr_param_grid, scoring = "accuracy")

In [59]:
%%time
lgr_results = test_classifiers(training_sets, lgr_grid)
print(lgr_results)

{'train_with_trend results': [0.5246674841996039, {'logisticregression__C': 0.001, 'logisticregression__penalty': 'l1'}, 0.524101499858504, 0.49697885196374625], 'train results': [0.5285350438637865, {'logisticregression__C': 0.0001, 'logisticregression__penalty': 'l2'}, 0.532968587869069, 0.5279456193353474]}
CPU times: user 21.5 s, sys: 1.34 s, total: 22.8 s
Wall time: 15.6 s


In [ ]:
lgr_feat = LogisticRegression

In [84]:
features_train.columns

Index(['1-hour-close-change-eth', '2-hour-close-change-eth',
       '3-hour-close-change-eth', '4-hour-close-change-eth',
       '6-hour-close-change-eth', '8-hour-close-change-eth',
       '10-hour-close-change-eth', '12-hour-close-change-eth', 'volume-eth',
       'range-eth', '1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'],
      dtype='object')

In [86]:
features_train_eth_only = features_train.drop(columns = ['1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'])

target_train_eth_only = target_train.drop(columns = ['1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'])

lgr_grid.fit(features_train_eth_only, target_train_eth_only)


0.5286293745873031

In [90]:
features_val_eth_only = features_val.drop(columns = ['1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'])

target_val_eth_only = target_val.drop(columns = ['1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'])

predictions = lgr_grid.predict(features_val_eth_only)
lgr_grid.score(features_val_eth_only, target_val_eth_only)

0.5098187311178247

#### Random Forest 

In [60]:
from sklearn.ensemble import RandomForestClassifier 

/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [61]:
rft_param_grid = {'randomforestclassifier__n_estimators': [250, 350, 450], 
                'randomforestclassifier__max_depth': [16, 20, 24]}

rft_pipe = make_pipeline(RandomForestClassifier())

rft_grid = GridSearchCV(rft_pipe, rft_param_grid, scoring = "accuracy")

In [62]:
%%time
rft_results = test_classifiers(training_sets, rft_grid)
print(rft_results)

{'train_with_trend results': [0.5262711064993868, {'randomforestclassifier__n_estimators': 450, 'randomforestclassifier__max_depth': 16}, 0.9864163758136025, 0.506797583081571], 'train results': [0.5339118951042354, {'randomforestclassifier__n_estimators': 350, 'randomforestclassifier__max_depth': 16}, 0.9678332232808226, 0.5151057401812689]}
CPU times: user 9min 7s, sys: 3.8 s, total: 9min 11s
Wall time: 9min 31s


In [73]:
rft_feat = RandomForestClassifier(n_estimators=350, max_depth=16)
rft_feat.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [74]:
rft_feat.feature_importances_

array([0.05608659, 0.05122262, 0.04892054, 0.04825953, 0.0492754 ,
       0.04694711, 0.04669115, 0.04899145, 0.04765615, 0.04775746,
       0.05505176, 0.05481912, 0.0504604 , 0.05071124, 0.05069636,
       0.04863547, 0.04803515, 0.049346  , 0.04970795, 0.05072854])

In [83]:
features_full.columns

Index(['1-hour-close-change-eth', '2-hour-close-change-eth',
       '3-hour-close-change-eth', '4-hour-close-change-eth',
       '6-hour-close-change-eth', '8-hour-close-change-eth',
       '10-hour-close-change-eth', '12-hour-close-change-eth', 'volume-eth',
       'range-eth', '1-hour-close-change-btc', '2-hour-close-change-btc',
       '3-hour-close-change-btc', '4-hour-close-change-btc',
       '6-hour-close-change-btc', '8-hour-close-change-btc',
       '10-hour-close-change-btc', '12-hour-close-change-btc', 'volume-btc',
       'range-btc'],
      dtype='object')

#### AdaBoost

In [63]:
from sklearn.ensemble import AdaBoostClassifier

In [64]:
ada = AdaBoostClassifier()

ada_param_grid = {'adaboostclassifier__n_estimators': [50, 150, 250, 350], 
                 'adaboostclassifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 1, 2]}

ada_pipe = make_pipeline(ada)

ada_grid = GridSearchCV(ada_pipe, ada_param_grid, scoring = "accuracy")

In [65]:
%%time
ada_results = test_classifiers(training_sets, ada_grid)
print(ada_results)

{'train_with_trend results': [0.5330629185925856, {'adaboostclassifier__n_estimators': 50, 'adaboostclassifier__learning_rate': 0.1}, 0.5439109517970003, 0.5188821752265861], 'train results': [0.534855202339402, {'adaboostclassifier__n_estimators': 250, 'adaboostclassifier__learning_rate': 0.01}, 0.5403263843033677, 0.5181268882175226]}
CPU times: user 8min 16s, sys: 2 s, total: 8min 18s
Wall time: 8min 31s


In [ ]:
ada_feat = AdaBoostClassifier()

#### Support Vector Machine

In [66]:
from sklearn.svm import SVC

In [67]:
svc = SVC()

svc_param_grid = {'svc__C' : [0.0001, 0.001, 0.01, 0.1, 1, 10], 
                 'svc__kernel' : ['rbf']}

svc_pipe = make_pipeline(svc)

svc_grid = GridSearchCV(svc_pipe, svc_param_grid, scoring = "accuracy")

In [68]:
%%time 
svc_results = test_classifiers(training_sets, svc_grid)
print(svc_results)

{'train_with_trend results': [0.5223092161116876, {'svc__C': 0.0001, 'svc__kernel': 'rbf'}, 0.5223092161116876, 0.49924471299093653], 'train results': [0.5223092161116876, {'svc__C': 0.0001, 'svc__kernel': 'rbf'}, 0.5223092161116876, 0.49924471299093653]}
CPU times: user 2min 41s, sys: 3.52 s, total: 2min 44s
Wall time: 2min 51s


#### Additional Boosting Models

<b> XGBoost </b>

In [69]:
import xgboost

In [70]:
xgb = xgboost.XGBClassifier()

xgb_param_grid = {'xgbclassifier__n_estimators' : [100, 200, 300], 
                 'xgbclassifier__max_depth' : [1, 3, 6, 9, 12], 
                 'xgbclassifier__learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 2]}

xgb_pipe = make_pipeline(xgb)

xgb_grid = GridSearchCV(xgb_pipe, xgb_param_grid, scoring = "accuracy")

In [71]:
%%time
xgb_results = test_classifiers(training_sets, xgb_grid)
print(xgb_results)

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

...

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use

{'train_with_trend results': [0.5350438637864352, {'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__max_depth': 1, 'xgbclassifier__n_estimators': 300}, 0.5405150457504009, 0.5181268882175226], 'train results': [0.5337232336572022, {'xgbclassifier__learning_rate': 0.01, 'xgbclassifier__max_depth': 1, 'xgbclassifier__n_estimators': 200}, 0.5401377228563343, 0.5181268882175226]}
CPU times: user 36min 12s, sys: 12.5 s, total: 36min 25s
Wall time: 38min 1s


/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.

/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



<b> Catboost </b>
Not yet implemented 

<b> Lightboost </b>

Not yet implemented 

In [ ]:
import lightgbm

# import lightgbm as lgb

In [ ]:
lgb = lightgbm.

lgb_param_grid = {'n_estimators' : [100, 200, 300], 
                 'max_depth' : [1, 3, 6, 9, 12], 
                 'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 2]}

lgb_pipe = make_pipeline(lgb)

lgb_grid = GridSearchCV(lgb_pipe, lgb_param_grid, scoring = "accuracy")

### Analyzing and Comparing Results

In [75]:
results = pd.DataFrame(columns = ["Model Type", "Train Accuracy", "Test Accuracy", "Train Accuracy with Trend Data", "Test Accuracy with Trend Data", "Time to Run"])

In [76]:
lgr_results_list = ['Logistic Regression', lgr_results['train results'][2], lgr_results['train results'][3], 
                    lgr_results['train_with_trend results'][2], lgr_results['train_with_trend results'][3], '00:16']
rft_results_list = ['Random Forest', rft_results['train results'][2], rft_results['train results'][3], 
                    rft_results['train_with_trend results'][2], rft_results['train_with_trend results'][3], '09:31']
ada_results_list = ['AdaBoost', ada_results['train results'][2], ada_results['train results'][3], 
                    ada_results['train_with_trend results'][2], ada_results['train_with_trend results'][3], '08:31']
svc_results_list = ['Support Vector Machine', svc_results['train results'][2], svc_results['train results'][3], 
                    svc_results['train_with_trend results'][2], svc_results['train_with_trend results'][3], '02:51']
xgb_results_list = ['XGBoost', xgb_results['train results'][2], xgb_results['train results'][3], 
                    xgb_results['train_with_trend results'][2], xgb_results['train_with_trend results'][3], '38:01']






In [77]:
rows = [lgr_results_list,
        rft_results_list,
        ada_results_list,
        svc_results_list, 
        xgb_results_list]
for row in rows:
    results.loc[len(results)] = row
    

In [78]:
results

,Model Type,Train Accuracy,Test Accuracy,Train Accuracy with Trend Data,Test Accuracy with Trend Data,Time to Run
0,Logistic Regression,0.532969,0.527946,0.524101,0.496979,00:16
1,Random Forest,0.967833,0.515106,0.986416,0.506798,09:31
2,AdaBoost,0.540326,0.518127,0.543911,0.518882,08:31
3,Support Vector Machine,0.522309,0.499245,0.522309,0.499245,02:51
4,XGBoost,0.540138,0.518127,0.540515,0.518127,38:01


In [79]:
results.to_csv('model_performance_data.csv', mode = "w+")

#### PCA

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(features_full)
feat_vectors_3d = pca.transform(features_full) # transform the vectors to 3d space
explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)[2]
print('Explained variance ratio is {:.2f}%, {:.2f}% is lost by reducing the dimensionality'.format((explained_variance_ratio*100), (100-(explained_variance_ratio*100)))) 

Explained variance ratio is 88.26%, 11.74% is lost by reducing the dimensionality


In [19]:
feat_vectors_3d_df = pd.DataFrame(feat_vectors_3d)
 = ['x', 'y', 'z']
feat_vectors_3d_df.head()

,x,y,z
0,5.048208,-1.978459,-16.201011
1,40.585746,12.272934,-8.935350
2,27.661819,4.513864,-14.970345
3,31.932778,2.632794,-18.433266
4,-2.610522,-15.670273,-24.951071


In [27]:
def color_map(category):
    cmap = plt.cm.viridis
    cmaplist = [cmap(i) for i in range(cmap.N)]
    jump = 256//len(category)
    cmaplist_ = [cmaplist[jump*i] for i in range(0,len(category))]
    return cmaplist_

features = feat_vectors_3d_df.columns

cmap_features = color_map(features)

In [32]:
feat_vectors_3d_df.columns.values

array(['x', 'y', 'z'], dtype=object)

In [40]:
pca_trace = go.Scatter3d(
            x = feat_vectors_3d_df['x'],
            y = feat_vectors_3d_df['y'],
            z = feat_vectors_3d_df['z'],
            mode='markers',
#             name = periods[trace['period'].values[0]],
            marker=dict(
                size=5,
#                 color= 'rgba' + str(cmap_periods[(trace['period'].values[0])]),
            )
            )

pca_data = [pca_trace]
plotly.offline.iplot(pca_data, validate=False, filename='pca_graph')